# Providing dynamic efficiency threshold as function

## Idea

We can give the efficiency threshold as a function with the signature:

```python
def eta_r(t,oe,mass,battery):
    # ... compute eta as a function of the inputs ...
    return eta_r
```

For example, this can be a function based on the battery level:

```python
def eta_r(t,oe,mass,battery):
    return min(400/(4*battery - battery_dod), 1)
```

## Demonstration

We first do our imports and define the initial/final state and weights


In [1]:
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("../../../")    # path to pyqlaw
import pyqlaw

In [2]:
# initial and final elements: [a,e,i,RAAN,omega,ta]
LU = 42164.0
GM_EARTH = 398600.44
VU = np.sqrt(GM_EARTH/LU)
TU = LU/VU

rp_gto = 200 + 6378
ra_gto = 35786 + 6378
sma_gto = (rp_gto + ra_gto)/(2*LU)
ecc_gto = (ra_gto - rp_gto)/(ra_gto + rp_gto)
KEP0 = [sma_gto,ecc_gto,np.deg2rad(23),0,0,0]
KEPF = [1,0,np.deg2rad(3),0,0,0]
oe0 = pyqlaw.kep2mee_with_a(np.array(KEP0))
oeT = pyqlaw.kep2mee_with_a(np.array(KEPF))
woe = [3.0, 1.0, 1.0, 1.0, 1.0]

# spacecraft parameters
MU = 1500
tmax_si = 1.0    # 1 N
isp_si  = 1500   # seconds
mdot_si = tmax_si/(isp_si*9.81)  # kg/s

# non-dimensional quantities
mass0 = 1.0
tmax = tmax_si * (1/MU)*(TU**2/(1e3*LU))
mdot = np.abs(mdot_si) *(TU/MU)
tf_max = 10000.0
t_step = np.deg2rad(15)

Suppose we want to define eta as a function of battery; we define some battery related quantities


In [3]:
# battery levels
battery_initial = 3000*3600/TU            # Wh --> Ws --> W.TU
battery_dod = 500*3600/TU
battery_capacity = (battery_dod,battery_initial)
charge_rate = 1500          # W
discharge_rate = 500        # W
battery_charge_discharge_rate = (charge_rate, discharge_rate)
require_full_recharge = True

We now construct the `QLaw` object, and set the problem with information about the battery. 

In [4]:
# construct problem
prob = pyqlaw.QLaw(
    integrator="rk4", 
    elements_type="mee_with_a",
    verbosity=2,
    print_frequency=500,
    use_sundman = True,
)

# set problem
prob.set_problem(oe0, oeT, mass0, tmax, mdot, tf_max, t_step,
    battery_initial = battery_initial,
    battery_capacity = battery_capacity,
    battery_charge_discharge_rate = battery_charge_discharge_rate,
    require_full_recharge = require_full_recharge,
    woe = woe)
prob.pretty()

Transfer:
  a  : 5.7800e-01 -> 1.0000e+00 (weight: 3.00)
  f  : 7.3009e-01 -> 0.0000e+00 (weight: 1.00)
  g  : 0.0000e+00 -> 0.0000e+00 (weight: 1.00)
  h  : 2.0345e-01 -> 2.6186e-02 (weight: 1.00)
  k  : 0.0000e+00 -> 0.0000e+00 (weight: 1.00)


We now define the functions `eta_a()` and `eta_r()` that will be called at each step of the integration. Here, we make use of the relative efficiency only, so the `eta_a()` function is merely a placeholder (we might as well pass `eta_a=0.0` later), but is still defined as a function for the sake of demonstration. 

In [5]:
# efficiency functions
def eta_a(t,oe,mass,battery):
    return 0.0    # not using absolute efficiency threshold

def eta_r(t,oe,mass,battery):
    return min(400/(4*battery - battery_dod), 1)

Finally, we call the `solve()` method of `prob`, passing the functions `eta_a` and `eta_r` as arguments

In [6]:
# solve
tstart_solve = time.time()
prob.solve(eta_a=eta_a, eta_r=eta_r)
prob.pretty_results()


 iter   |  time      |  del1       |  del2       |  del3       |  del4       |  del5       |  el6        |
      0 |  3.201e-02 | -4.2200e-01 |  7.3009e-01 |  0.0000e+00 |  1.7727e-01 |  0.0000e+00 |  1.9244e-01 |
    500 |  5.439e+01 | -3.8749e-01 |  6.7680e-01 |  3.2627e-02 |  1.3495e-01 |  7.6635e-03 |  1.3391e+02 |
   1000 |  1.147e+02 | -3.4718e-01 |  6.1650e-01 |  6.3203e-02 |  9.4574e-02 |  1.3576e-02 |  2.5466e+02 |
   1500 |  1.813e+02 | -2.8014e-01 |  5.3557e-01 |  9.1325e-02 |  5.4481e-02 |  1.4602e-02 |  3.6952e+02 |
   2000 |  2.604e+02 | -1.7930e-01 |  4.2520e-01 |  1.0199e-01 |  2.4264e-02 |  1.2922e-02 |  4.8334e+02 |
   2500 |  3.649e+02 | -1.0009e-02 |  2.3379e-01 |  8.4460e-02 |  6.2573e-03 |  7.9227e-03 |  5.9925e+02 |
Target elements successfully reached!
Exit code : 2
Converge  : True
Final state:
  a  : 9.9974e-01 (error: 2.6158e-04)
  f  : 6.1971e-05 (error: 6.1971e-05)
  g  : 5.3825e-03 (error: 5.3825e-03)
  h  : 2.6128e-02 (error: 5.7767e-05)
  k  : 8.4103e-0

and we can visualize the results

In [ ]:
# plots
fig1, ax1 = prob.plot_elements_history(to_keplerian=True, TU=TU/86400, time_unit_name="day")
fig2, ax2 = prob.plot_trajectory_3d(sphere_radius=0.1)
fig3, ax3 = prob.plot_controls()
fig4, ax4 = prob.plot_battery_history(TU=TU/86400, BU=TU/3600,
    time_unit_name="day", battery_unit_name="Wh")
fig5, ax5 = prob.plot_efficiency(TU=TU/86400, time_unit_name="day")
plt.show()